In [1]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
%%time

# Init script
def init_script():
    file_path = os.path.join(OUTPUT_FOLDER, '107_FRANCE.csv')
    if path.exists(file_path):
        df_check = pd.read_csv(file_path, sep=';', low_memory=False)
        date_check = df_check[df_check['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y')]
        if len(date_check) > 0:
            print(f'Data already updated !')
            %stop

init_script()

CPU times: user 6.83 ms, sys: 688 µs, total: 7.52 ms
Wall time: 6.6 ms


In [4]:
%%time

# Variables
bob_statics = ''

# Load Data Source
def load_source():
    file_path = os.path.join(OUTPUT_FOLDER,'107.csv')
    df = pd.DataFrame()
    
    if path.exists(file_path):
        df = pd.read_csv(file_path,sep=";")
        df = df.drop('Unnamed: 0',axis=1)
        df = df[(df['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))].reset_index(drop=True)
        if len(df) == 0:
            print(f'Data not updated !')
            %stop
        else:
            df_save(df,'107_FRANCE','csv')
    return df


# Trasnform Data Source
def transform_data(df):
    if not df.empty:
        df = pd.merge(df, REF_FRANCE, left_on='CHILD', right_on='DEP_FULL', how='left'
                     ).rename(index=str, columns={"DEP_NAME":"LABEL"}
                             ).drop(['DEP_CODE','DEP_FULL','REG_NAME'], axis=1)

        df.loc[df['PARENT'] == 'root', 'LABEL'] = df['CHILD']
        df = df.drop(['UNIT_VARP','PRECISION_VALUE',
                      'PRECISION_VAR','PRECISION_VARP',
                      'LAST_UPDATE'], axis=1)
    else:
        print(f'DataFrame empty !')
    return df

df_france = transform_data(load_source())
df_france.head(5)

107_FRANCE successfully saved in 107_FRANCE
CPU times: user 109 ms, sys: 19.3 ms, total: 128 ms
Wall time: 130 ms


,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,LABEL
0,France,11/06/2020,20200611,Île-de-France,root,En réanimation,2.0,428,-7,-1.609195,Île-de-France
1,France,11/06/2020,20200611,Île-de-France,root,Hospitalisations,1.0,5150,-86,-1.642475,Île-de-France
2,France,11/06/2020,20200611,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,44,0,0.000000,Haute-Garonne
3,France,11/06/2020,20200611,32 - Gers,Occitanie,Hospitalisations,1.0,3,0,0.000000,Gers
4,France,11/06/2020,20200611,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,87,-2,-2.247191,Gironde


In [5]:
#Var France
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map(df,geo,dfKey,geoKey,scale,colors,map_center,label,file_html):
    file_path = os.path.join(OUTPUT_FOLDER,file_html)
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color="VALUE",
                               locations=dfKey, 
                               featureidkey=geoKey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data=["VALUE","VAR"],
                               labels=label,
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(file_path)
    updateChartCss(file_path, os.path.join(INPUT_FOLDER,'map.css'))
#     return fig

In [6]:
# MAP_WSR = pd.read_excel(REF_WSR_PATH, sheet_name="MAP_FRANCE")

# def generate_map(df):
#     for _, row in MAP_WSR.iterrows():
#         kpi = row['KPI']
#         print(kpi)
#         file_html = row['FILE_HTML']
#         print(file_html)
#         scale_min = row['SCALE_MIN']
#         scale_max = row['SCALE_MAX']
#         scale = (scale_min, scale_max)
#         print(scale)
#         colors_s1 = row['COLORS_S1']
#         colors_c1 = row['COLORS_C1']
#         colors_s2 = row['COLORS_S2']
#         colors_c2 = row['COLORS_C2']
#         colors_s3 = row['COLORS_S3']
#         colors_c3 = row['COLORS_C3']
#         colors_s4 = row['COLORS_S4']
#         colors_c4 = row['COLORS_C4']
#         colors_s5 = row['COLORS_S5']
#         colors_c5 = row['COLORS_C5']
#         colors = [[colors_s1,colors_c1],[colors_s2,colors_c2],[colors_s3,colors_c3],[colors_s4,colors_c4],[colors_s5,colors_c5]]
#         print(colors)
#         dfKey = row['DF_KEY']
#         print(dfKey)
#         geoKey = row['GEO_KEY']
#         print(geoKey)
#         to_rename = row['TO_RENAME']
#         print(to_rename)
        
#         df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
#         analysis(df_map,'VALUE')
        
#         create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
#         print(token_hosp_dep)
        
# generate_map(df_france)

In [7]:
#Hospitalisation - Département
kpi = 'Hospitalisations'
file_html = "France_HOSP_DEP_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_hosp_dep = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 0, Max: 874, Average: 113.0, AvrMax: 0.13, Med: 40.0, MedMax: 0.05
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "6567ea3baac3e16e9cc43f7aae7e79cde6b96218d958e6a120ea88f702de", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_HOSP_DEP_NAME.html", "filename": "../data_output/France_HOSP_DEP_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/6567ea3baac3e16e9cc43f7aae7e79cde6b96218d958e6a120ea88f702de 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [8]:
#En réanimation - Département
kpi = 'En réanimation'
file_html = "France_REA_DEP_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,800)
colors = [
    [0.0, "#FFFFFF"],
    [0.025,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_rea_dep = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 0, Max: 112, Average: 9.0, AvrMax: 0.08, Med: 3.0, MedMax: 0.03
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "aac67f546d15c4f0a1070e4d730dadf77861de62e9ff5658c08462b4c5ad", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_REA_DEP_NAME.html", "filename": "../data_output/France_REA_DEP_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/aac67f546d15c4f0a1070e4d730dadf77861de62e9ff5658c08462b4c5ad 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [9]:
#"Décès à l’hôpital" - Département
kpi = "Décès à l’hôpital"
file_html = "France_DC_DEP_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,1500)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_dc_dep = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 1, Max: 1715, Average: 188.0, AvrMax: 0.11, Med: 80.0, MedMax: 0.05
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "2d3912699ade4b262cf6e39a5d6a3362d95d1e1a1dd42b32b00a7b6cd4b9", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_DC_DEP_NAME.html", "filename": "../data_output/France_DC_DEP_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/2d3912699ade4b262cf6e39a5d6a3362d95d1e1a1dd42b32b00a7b6cd4b9 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [10]:
#'Retours à domicile' - Département
kpi = 'Retours à domicile'
file_html = "France_RAD_DEP_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_DEP,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_rad_dep = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 24, Max: 5833, Average: 714.0, AvrMax: 0.12, Med: 290.0, MedMax: 0.05
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "43a8bd5eedba3dac8599db08b05e31fd25cba4b6a32c151e1cff23f7b894", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_RAD_DEP_NAME.html", "filename": "../data_output/France_RAD_DEP_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/43a8bd5eedba3dac8599db08b05e31fd25cba4b6a32c151e1cff23f7b894 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


#ANALYSIS BY REGION

In [11]:
#Hospitalisation - Région
kpi = 'Hospitalisations'
file_html = "France_HOSP_REG_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] == 'root') & (df_france['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,13000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_REG,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_hosp_reg = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 10, Max: 5150, Average: 634.0, AvrMax: 0.12, Med: 222.0, MedMax: 0.04
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "962c5d6f49ee7370f66ca428fe3dfe6d8645e6d7b4975ffcda4e413960dd", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_HOSP_REG_NAME.html", "filename": "../data_output/France_HOSP_REG_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/962c5d6f49ee7370f66ca428fe3dfe6d8645e6d7b4975ffcda4e413960dd 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [12]:
#En réanimation - Région
kpi = 'En réanimation'
file_html = "France_REA_REG_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] == 'root') & (df_france['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (50,2500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_REG,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_rea_reg = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 0, Max: 428, Average: 49.0, AvrMax: 0.11, Med: 16.0, MedMax: 0.04
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "bdbd5025962ec4d3ce2831ef754611881d65c4230c9d44fc0d51d263866f", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_REA_REG_NAME.html", "filename": "../data_output/France_REA_REG_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/bdbd5025962ec4d3ce2831ef754611881d65c4230c9d44fc0d51d263866f 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [13]:
#"Décès à l’hôpital" - Région
kpi = "Décès à l’hôpital"
file_html = "France_DC_REG_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] == 'root') & (df_france['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,5500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_REG,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_dc_reg = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 1, Max: 7295, Average: 1052.0, AvrMax: 0.14, Med: 442.0, MedMax: 0.06
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "f6949fd81061acbeca89e03bcf30e1e0135ecc3a8d370da676c79aa95288", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_DC_REG_NAME.html", "filename": "../data_output/France_DC_REG_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/f6949fd81061acbeca89e03bcf30e1e0135ecc3a8d370da676c79aa95288 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [14]:
#Retours à domicile - Région
kpi = 'Retours à domicile'
file_html = "France_RAD_REG_NAME.html"

df_map = df_france.copy()[(df_france['UPPER_FILTER_M'] == kpi) & (df_france['PARENT'] == 'root') & (df_france['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (200,15000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,MAP_FRANCE_REG,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_rad_reg = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

VALUE : Min: 74, Max: 26083, Average: 4008.0, AvrMax: 0.15, Med: 1876.0, MedMax: 0.07
[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "f37f4a83ee7445ad03abffdbfe6ee3a1d1a181168e683008aaf2932d78ec", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_RAD_REG_NAME.html", "filename": "../data_output/France_RAD_REG_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/f37f4a83ee7445ad03abffdbfe6ee3a1d1a181168e683008aaf2932d78ec 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [15]:
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map_epid(df,geo,dfKey,geoKey,col_color,hoverdata,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color=col_color,
                               locations=dfKey, 
                               featureidkey=geoKey,
                                color_discrete_map={ # replaces default color mapping by value
                                    "vert": "#4CCD88", "orange": "#FDB47A", "rouge": "#DD6D88"
                                },
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data= hoverdata,
                               labels=label,
                               opacity=0.8
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True, showlegend=False)
    fig.write_html(os.path.join(OUTPUT_FOLDER,file_html))
    updateChartCss(os.path.join(OUTPUT_FOLDER,file_html), os.path.join(INPUT_FOLDER,'map.css'))
#     return fig

In [16]:
#--- Situation epidémiologique DATA
cols = ['DEP','DEP_NAME','REA_2013','REA_2018','SI_2013','SI_2018','SC_2013','SC_2018','ALL_2013','ALL_2018']
ref_france_lits = pd.read_excel(os.path.join(INPUT_FOLDER,'drees_lits_reanimation_2013-2018.xlsx'), sheet_name = 'Lits_2013-2018', header=7, names=cols)
ref_france_lits = ref_france_lits[['DEP','DEP_NAME','REA_2018']]
ref_france_lits['DEP'] = ref_france_lits['DEP'].astype(str)
ref_france_lits.head(5)

,DEP,DEP_NAME,REA_2018
0,FR,France entière,5432
1,MET,France métropolitaine,5290
2,01,Ain,22
3,02,Aisne,34
4,03,Allier,24


In [17]:
#---Tension Data
df_rea = pd.read_csv(os.path.join(OUTPUT_FOLDER,'107_FRANCE.csv'),sep=";").drop('Unnamed: 0',axis=1)
df_rea = df_rea[['ENTITY','SCENARIO','CHILD','PARENT','UPPER_FILTER_M','VALUE','VAR']]
df_rea = pd.merge(df_rea, REF_FRANCE, left_on='CHILD', right_on='DEP_FULL',how='left').rename(index=str, columns={"DEP_NAME":"LABEL"}).drop(['DEP_FULL'], axis=1)
df_rea.loc[df_rea['PARENT'] == 'root', 'LABEL'] = df_rea['CHILD']
df_rea['DEP_CODE'] = df_rea['DEP_CODE'].astype(str)

df_rea.loc[df_rea['CHILD'] == 'France', 'DEP_CODE'] = 'FR'
df_rea = pd.merge(df_rea,ref_france_lits, left_on='DEP_CODE', right_on='DEP',how='left').drop(['DEP','DEP_NAME'], axis=1)
df_rea.head(5)

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018
0,France,11/06/2020,Île-de-France,root,En réanimation,428,-7,nan,Île-de-France,NaN,NaN
1,France,11/06/2020,Île-de-France,root,Hospitalisations,5150,-86,nan,Île-de-France,NaN,NaN
2,France,11/06/2020,31 - Haute-Garonne,Occitanie,Hospitalisations,44,0,31,Haute-Garonne,Occitanie,165.0
3,France,11/06/2020,32 - Gers,Occitanie,Hospitalisations,3,0,32,Gers,Occitanie,8.0
4,France,11/06/2020,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,87,-2,33,Gironde,Nouvelle-Aquitaine,195.0


In [18]:
#---HKPIS
df_hkpis = df_rea.copy()[(df_rea['CHILD'] == 'France') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
df_kpis = df_hkpis[['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR']]

df_reakpi = df_hkpis.copy()[df_hkpis['UPPER_FILTER_M'] == 'En réanimation'][['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR','REA_2018']]
df_reakpi['VALUEP'] = round((df_reakpi['VALUE'] / df_reakpi['REA_2018'] * 100),2)
df_reakpi['VALUE_N1'] = (df_reakpi['VALUE'] - df_reakpi['VAR'])
df_reakpi['VALUEP_N1'] = round(((df_reakpi['VALUE_N1'] / df_reakpi['REA_2018']) * 100),2)
df_reakpi['VALUEP_VAR'] = df_reakpi['VALUEP'] - df_reakpi['VALUEP_N1']

df_reakpi = df_reakpi.drop(['VALUE','VAR','REA_2018','VALUE_N1','VALUEP_N1'],axis=1).rename(index=str, columns={"VALUEP":"VALUE","VALUEP_VAR":"VAR"})
df_reakpi['UPPER_FILTER_M'] = 'Tensions hospitalières'

df_final = pd.concat([df_kpis,df_reakpi], axis=0)
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VALUE'] = ' %'
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VAR'] = ' pts'
df_final.loc[:,'PRECISION'] = ',.0f'
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','PRECISION'] = ',.2f'

if USE_MONGO:
    bob.mongo.save_df(df_final,'110_BIS',DB_APP,True)
df_final

Dataframe 110_BIS successfully save in database app-wsr in MongoDB. Time: --- 0.021678686141967773 secnds ---


,ENTITY,SCENARIO,UPPER_FILTER_M,VALUE,VAR,UNIT_VALUE,UNIT_VAR,PRECISION
77,France,11/06/2020,Hospitalisations,11420.00,-212.00,NaN,NaN,",.0f"
199,France,11/06/2020,Décès à l’hôpital,18942.00,27.00,NaN,NaN,",.0f"
316,France,11/06/2020,En réanimation,875.00,-29.00,NaN,NaN,",.0f"
436,France,11/06/2020,Retours à domicile,72139.00,317.00,NaN,NaN,",.0f"
316,France,11/06/2020,Tensions hospitalières,16.11,-0.53,%,pts,",.2f"


In [19]:
#Par département
df_rea_dep = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
df_rea_dep['VALUEP'] = round((df_rea_dep['VALUE'] / df_rea_dep['REA_2018'] * 100),2)
df_rea_dep.loc[df_rea_dep['VALUEP'] < 60 ,'COLOR'] = 'vert'
df_rea_dep.loc[df_rea_dep['VALUEP'] >= 60, 'COLOR'] = 'orange'
df_rea_dep.loc[df_rea_dep['VALUEP'] >= 80, 'COLOR'] = 'rouge'
df_rea_dep.head(5)

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018,VALUEP,COLOR
242,France,11/06/2020,01 - Ain,Auvergne-Rhône-Alpes,En réanimation,0,0,01,Ain,Auvergne-Rhône-Alpes,22.0,0.00,vert
243,France,11/06/2020,32 - Gers,Occitanie,En réanimation,1,0,32,Gers,Occitanie,8.0,12.50,vert
244,France,11/06/2020,33 - Gironde,Nouvelle-Aquitaine,En réanimation,11,0,33,Gironde,Nouvelle-Aquitaine,195.0,5.64,vert
245,France,11/06/2020,34 - Hérault,Occitanie,En réanimation,2,-1,34,Hérault,Occitanie,128.0,1.56,vert
246,France,11/06/2020,35 - Ille-et-Vilaine,Bretagne,En réanimation,3,0,35,Ille-et-Vilaine,Bretagne,66.0,4.55,vert


In [20]:
#---Tension_Département
kpi = 'Tension hospitalière (%)'
file_html = "France_TSHOSP_DEP_NAME.html"
dfKey = "LABEL"
geoKey="properties.nom"
col_color = 'COLOR'
hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
to_rename = {'COLOR': "Niveau",'LABEL':'Département', 'VALUE': 'Nombre de cas en réanimation', 'VAR': 'Evolution en réanimation','REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
create_map_epid(df_rea_dep,MAP_FRANCE_DEP,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_th_dep = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "b4d9267c00b5537d42dc0e9fa8c3c32eb9ca9c8ace56bc346b2e1d4427a5", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_TSHOSP_DEP_NAME.html", "filename": "../data_output/France_TSHOSP_DEP_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/b4d9267c00b5537d42dc0e9fa8c3c32eb9ca9c8ace56bc346b2e1d4427a5 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [21]:
#Par région
df_rea_re = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == YESTERDAY.strftime('%d/%m/%Y'))]
cols_to_group = ['ENTITY','SCENARIO','REG_NAME']
df_rea_re['VALUE'] = df_rea_re['VALUE'].astype(int)
df_rea_re['VAR'] = df_rea_re['VAR'].astype(int)
df_rea_re = df_rea_re.groupby(cols_to_group, as_index=False).agg({"VALUE":'sum',"VAR":'sum',"REA_2018":'sum'})

df_rea_re['VALUEP'] = round((df_rea_re['VALUE'] / df_rea_re['REA_2018'] * 100),2)
df_rea_re.loc[df_rea_re['VALUEP'] < 60 ,'COLOR'] = 'vert'
df_rea_re.loc[df_rea_re['VALUEP'] >= 60, 'COLOR'] = 'orange'
df_rea_re.loc[df_rea_re['VALUEP'] >= 80, 'COLOR'] = 'rouge'
df_rea_re.head(5)

,ENTITY,SCENARIO,REG_NAME,VALUE,VAR,REA_2018,VALUEP,COLOR
0,France,11/06/2020,Auvergne-Rhône-Alpes,50,-14,609.0,8.21,vert
1,France,11/06/2020,Bourgogne-Franche-Comté,22,0,207.0,10.63,vert
2,France,11/06/2020,Bretagne,11,0,171.0,6.43,vert
3,France,11/06/2020,Centre-Val de Loire,30,0,191.0,15.71,vert
4,France,11/06/2020,Corse,6,0,18.0,33.33,vert


In [22]:
#---Tension_Région
kpi = 'Tension hospitalière (%)'
file_html = "France_TSHOSP_REG_NAME.html"
dfKey = "REG_NAME"
geoKey="properties.nom"
col_color = 'COLOR'
hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
to_rename = {'REG_NAME':'Région','COLOR': "Niveau", 'VALUE': 'Nombre de cas en réanimation','VAR': 'Evolution en réanimation', 'REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
create_map_epid(df_rea_re,MAP_FRANCE_REG,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)

#Make html visible
bob_static = bob.makeStaticGettable(os.path.join(OUTPUT_FOLDER,file_html),"dev")
token_th_reg = bob_static[24:84]
bob_statics = f"{bob_statics}{bob_static}\n"

[Bob from JupyterHub] => Add this line: 
 # bob.static {"token": "1442e47182ce1b324c59e7fa35e9b01f6ac441d04f9dde8482c90d3acb8e", "path": "/home/ftp/ftp/prod/wsr-public/data_output/France_TSHOSP_REG_NAME.html", "filename": "../data_output/France_TSHOSP_REG_NAME.html"} 

At the top of this file to make it gettable at this url 
 https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/1442e47182ce1b324c59e7fa35e9b01f6ac441d04f9dde8482c90d3acb8e 

Now you can remove the code bob.makeStaticGettable() from this file
If you want to remove the gettable capability, just remove the line you have added (# bob.static ...) or the file


In [23]:
# html
# Par département
RACINE = 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/'

HOSP_DEP = [{'URL': (RACINE + token_hosp_dep),
             'KPI': 'Hospitalisations',
             'UPPER_FILTER': 'Par département'}]

REA_DEP = [{'URL': (RACINE + token_rea_dep),
            'KPI': 'En réanimation',
            'UPPER_FILTER': 'Par département'}]

DC_DEP = [{'URL': (RACINE + token_dc_dep),
           'KPI': 'Retours à domicile',
           'UPPER_FILTER': 'Par département'}]

RAD_DEP = [{'URL': (RACINE + token_rad_dep),
            'KPI': 'Décès à l’hôpital',
            'UPPER_FILTER': 'Par département'}]

REAT_DEP = [{'URL': (RACINE + token_th_dep),
             'KPI': 'Tensions hospitalières',
             'UPPER_FILTER': 'Par département'}]

# Par région
HOSP_REG = [{'URL': (RACINE + token_hosp_reg),
             'KPI': 'Hospitalisations',
             'UPPER_FILTER': 'Par région'}]

REA_REG = [{'URL': (RACINE + token_rea_reg),
            'KPI': 'En réanimation',
            'UPPER_FILTER': 'Par région'}]

DC_REG = [{'URL': (RACINE + token_dc_reg),
           'KPI': 'Retours à domicile',
           'UPPER_FILTER': 'Par région'}]

RAD_REG = [{'URL': (RACINE + token_rad_reg),
            'KPI': 'Décès à l’hôpital',
            'UPPER_FILTER': 'Par région'}]

REAT_REG = [{'URL': (RACINE + token_th_reg),
             'KPI': 'Tensions hospitalières',
             'UPPER_FILTER': 'Par région'}]

#Concat list to create Series
LIST_KPIS = (HOSP_DEP + REA_DEP + DC_DEP + RAD_DEP +
             HOSP_REG + REA_REG + DC_REG + RAD_REG + 
             REAT_DEP + REAT_REG)

#Create Dataframe
DOMAIN = pd.DataFrame(LIST_KPIS)
DOMAIN.loc[:, 'VALUE'] = ("<iframe src='" +
                          DOMAIN['URL'] +
                          ("' frameborder='0' style='overflow:hidden;"
                           "height:85vh;width:100%' height='100%' "
                           "width='100%'></iframe> <style>.modebar"
                           "{display: none!important;}</style>"))
if USE_MONGO:
    bob.mongo.save_df(DOMAIN,'110',DB_APP,True)
DOMAIN

Dataframe 110 successfully save in database app-wsr in MongoDB. Time: --- 0.023787975311279297 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par département,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par département,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par département,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par département,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par région,<iframe src='https://api.dev.cashstory.com/api...
5,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par région,<iframe src='https://api.dev.cashstory.com/api...
6,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par région,<iframe src='https://api.dev.cashstory.com/api...
7,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par région,<iframe src='https://api.dev.cashstory.com/api...
8,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par département,<iframe src='https://api.dev.cashstory.com/api...
9,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par région,<iframe src='https://api.dev.cashstory.com/api...


In [24]:
file = open(os.path.join(OUTPUT_FOLDER,"map_chart_france_statics.ipynb"),'w')
file.write(bob_statics) 
file.close() 